In [ ]:
## getting nessecary imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")

In [ ]:
application_df = application_df.drop(['EIN','NAME','APPLICATION_TYPE'], axis = 1)
application_df.head(3)

KeyError: ignored

In [ ]:
## resuing the same cutoff value from previous model in repository
count_apps = application_df.APPLICATION_TYPE.value_counts()

#lowered the app types to replace from 6 to 4 to improve accuracy
application_types_to_replace = list(count_apps[count_apps<6].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# check for successful binning
application_df['APPLICATION_TYPE'].value_counts()

AttributeError: ignored

In [ ]:
## using same cutoff value from previous model in repository
vc_c = application_df['CLASSIFICATION'].value_counts()
class_count = vc_c[vc_c>1]

#lowered the app types to replace from 6 to 4 to improve accuracy
classifications_to_replace = list(class_count[class_count<4].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# check for successful binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C4500        1
C2561        1
C2150        1
Name: CLASSIFICATION, Length: 64, dtype: int64

In [ ]:
application_df = pd.get_dummies(application_df, dtype=float)
application_df.head(3)

,STATUS,ASK_AMT,IS_SUCCESSFUL,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,CLASSIFICATION_C1000,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
y= application_df['IS_SUCCESSFUL'].values
X= application_df.drop(['IS_SUCCESSFUL'], axis =1).values

In [ ]:
# switched random state to 50 from previous 42
# switchedtest_size from 0.3 to 0.5 from previous model
X_train, X_test, y_train, y_test = train_test_split(
  X, y, test_size = 0.5, random_state=50)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Building the model


In [ ]:
# Defining model
number_input_features = len( X_train_scaled[0])

nn_model = tf.keras.models.Sequential()

## Hidden layers
# I've expanded the hidden layers from 7 to 21
nn_model.add(tf.keras.layers.Dense(units=21, activation="relu", input_dim=number_input_features))
nn_model.add(tf.keras.layers.Dense(units=21, activation="relu", input_dim=21))
nn_model.add(tf.keras.layers.Dense(units=21, activation="sigmoid", input_dim=21))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid", input_dim = 3))

#model summary
nn_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 21)                1953      
                                                                 
 dense_21 (Dense)            (None, 21)                462       
                                                                 
 dense_22 (Dense)            (None, 21)                462       
                                                                 
 dense_23 (Dense)            (None, 1)                 22        
                                                                 
Total params: 2,899
Trainable params: 2,899
Non-trainable params: 0
_________________________________________________________________


In [ ]:
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
#training model with same number of epochs
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
536/536 [==============================] - 4s 4ms/step - loss: 0.6059 - accuracy: 0.6827
Epoch 2/100
536/536 [==============================] - 2s 4ms/step - loss: 0.5785 - accuracy: 0.7056
Epoch 3/100
536/536 [==============================] - 3s 5ms/step - loss: 0.5744 - accuracy: 0.7082
Epoch 4/100
536/536 [==============================] - 2s 4ms/step - loss: 0.5725 - accuracy: 0.7100
Epoch 5/100
536/536 [==============================] - 2s 4ms/step - loss: 0.5709 - accuracy: 0.7134
Epoch 6/100
536/536 [==============================] - 2s 3ms/step - loss: 0.5701 - accuracy: 0.7133
Epoch 7/100
536/536 [==============================] - 1s 2ms/step - loss: 0.5700 - accuracy: 0.7144
Epoch 8/100
536/536 [==============================] - 1s 2ms/step - loss: 0.5686 - accuracy: 0.7153
Epoch 9/100
536/536 [==============================] - 1s 2ms/step - loss: 0.5683 - accuracy: 0.7143
Epoch 10/100
536/536 [==============================] - 1s 2ms/step - loss: 0.5683 - accura

In [ ]:
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
from google.colab import files
nn_model.save('AlphabetSoupCharity_Optimized.h5')
files.download("AlphabetSoupCharity_Optimized.h5")